In [1]:
import torch
nn = torch.nn
import torch.nn.functional as F
import numpy as np

In [9]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=1)
src = torch.rand(11, 32, 512)
out = encoder_layer(src)

In [11]:
for k in encoder_layer.state_dict():
  print(k)

self_attn.in_proj_weight
self_attn.in_proj_bias
self_attn.out_proj.weight
self_attn.out_proj.bias
linear1.weight
linear1.bias
linear2.weight
linear2.bias
norm1.weight
norm1.bias
norm2.weight
norm2.bias


In [13]:
embed_dim = 128
src_seq_len = 7
target_seq_len = 11
multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=1)
query = torch.rand(target_seq_len, embed_dim)
key = torch.rand(src_seq_len, embed_dim)
value = torch.rand(src_seq_len, embed_dim)
attn_output, _ = multihead_attn(query, key, value)

In [15]:
attn_output.shape

torch.Size([11, 128])

In [16]:
multihead_attn.state_dict()

OrderedDict([('in_proj_weight',
              tensor([[ 0.0758,  0.0992,  0.0953,  ...,  0.1030,  0.0789,  0.0251],
                      [ 0.0566, -0.0005,  0.0144,  ..., -0.0931,  0.0464,  0.0776],
                      [ 0.0438, -0.0998,  0.0972,  ...,  0.1082,  0.1075, -0.1067],
                      ...,
                      [ 0.1020,  0.0346,  0.0452,  ..., -0.0749,  0.0470, -0.0238],
                      [-0.0723,  0.1050,  0.0216,  ...,  0.0568, -0.0187,  0.0035],
                      [-0.0452, -0.0472, -0.0731,  ...,  0.1009,  0.0142,  0.0279]])),
             ('in_proj_bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 

In [ ]:
# https://github.com/pytorch/examples/blob/main/word_language_model/model.py
# https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html
# https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html

input_dim = 128
key_dim = 128
value_dim = 128
input_layer = nn.sequential(
    nn.Linear(input_dim, 2*key_dim + value_dim),
    nn.Relu(),
)
k_layer = nn.Linear(key_dim, key_dim)
q_layer = nn.Linear(key_dim, key_dim)
v_layer = nn.Linear(value_dim, value_dim)

x = torch.rand(input_dim)
x = input_layer(x)
k, q, v = torch.split(x, [key_dim, key_dim, value_dim])
k = k_layer(k)
q = q_layer(q)
# etc

In [18]:
l = nn.Linear(3,4)
x = torch.rand(3)
l(x)

tensor([ 0.6354, -0.4301,  0.7350,  0.2713], grad_fn=<AddBackward0>)

In [15]:
# my self attention
class MySelfAttention(nn.Module):
  def __init__(self, ):
    super(MySelfAttention, self).__init__()

  def forward(self, ks, qs, vs):
    '''
    seq_len = ks.size()[0]
    qdim = qs.size()[1]
    vdim = vs.size()[1] # = ks.size()[1]
    '''
    
    qk_matrix = qs @ torch.t(ks)
    return F.softmax(qk_matrix, dim=1) @ vs # softmax done to the rows

 

In [16]:
# below is an example of how the forward function of the MySelfAttention class works:
seq_len = 10
key_dim = 64
value_dim = 128
ks = torch.rand(seq_len, key_dim)
qs = torch.rand(seq_len, key_dim)
vs = torch.rand(seq_len, value_dim)

qk_matrix = qs @ torch.t(ks)
print(qk_matrix.size())

P = F.softmax(qk_matrix, dim=1)
# the following should be 1
print(torch.sum(P[1, :]))
print(torch.sum(P[2, :]))

A = P @ vs
print(A.shape) # should have shape (seq_len, value_dim)

torch.Size([10, 10])
tensor(1.)
tensor(1.0000)
torch.Size([10, 128])


In [18]:
att = MySelfAttention()
att(ks, qs, vs).size()

torch.Size([10, 128])